# we downloaded 100 datasets from pangalodb..

- some are quite irregular 
- we still want at least 4 representatives per tissuetype


## TODO

1.  make the table the old way
2.  switch to natto
3.  add status for the 3 levels of unknown 

In [ ]:
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = True

import matplotlib as mpl
mpl.rc("figure", dpi=200)

In [ ]:
import seaborn as sns
import pandas as pd
from cellsaw import load as loader
import lmz

labels = loader.get100names()
shortlabels = [label[:label.index('_')] for label in labels]
loadparams = {'path' : '/home/ubuntu/repos/natto/natto/data',
              'remove_unlabeled':False, 'mingenes': 200, 
              'preprocessingmethod':'natto', "subsample":1000}

def load(names):
    # load 100 adata will have:
    # "true" cluster annotation
    # 'name' dataset name 
    adatas = [loader.easyLoad100(x,**loadparams) for x in names]
    # this will add the true celltype labels for the pangalodb
    # 'truecelltyp'
    return loader.loadadata.annotatepangalo(loadparams['path'], adatas, names)[0]

def getdata(dataset):
    adata = load([dataset])
    le  = adata.X.shape[0]
    m = sum(adata.obs['true'] == -1)
    c = sum(adata.obs['truecelltype'] == "pangalofail")
    u = sum(adata.obs['truecelltype'] == "Unknown")
    return [(dataset,'true -1', m/le ) , 
            (dataset,'cluster fail',c/le),
            (dataset, 'Unknown', u/le),
           (dataset, 'total', (c+u)/le)]
    
data = [a for b in map(getdata, labels) for a in b]



In [ ]:
labels, types, values = lmz.Transpose(data)
data = {"dataset":labels, "type": types, "percentage": values}

df  = pd.DataFrame.from_dict(data)
sns.barplot(x="dataset", y="percentage", hue="type", data=df)

In [ ]:
list(df[df['type']=='total'][df['percentage'] < .7]['dataset'])

In [ ]:
print(df[df['type']=='total'][df['percentage'] < .7].to_markdown())

In [ ]:
len(df[df['type']=='total'][df['percentage'] < .7])

In [ ]:
%time 
# load the data
from natto import input 
from natto import process
from ubergauss import tools


names = input.get100names()

def load_and_score(dataset_name):
    data = input.load100(dataset_name,path = '/home/ubuntu/repos/natto/natto/data')
    gs= process.annotate_genescores(data, mingenes = 200)
    gs.X = np.expm1(gs.X)
    return gs 

annotated = tools.xmap(load_and_score,names)

#data = [input.load100(n,path = '/home/ubuntu/repos/natto/natto/data')  for n in names]
#annotated = [process.annotate_genescores(d, mingenes = 1000) for d in data]  

In [ ]:
# so we collect stats and put them in a table 



from matplotlib import pyplot as plt
import numpy as np
from lmz import *
import seaborn as sns 
import pandas as pd 
pd.options.display.max_rows = 1000

data = {'names': names}


# lets say we need 2k annotated cells
data['annotated cells']=[ sum(a.obs['true']!=-1) for a in annotated]
data['diverse cells'] = [ np.sum( np.array([ len(row.indices) for row in a.X]) > 1000 ) for a in annotated]
data['numgoodgenes'] = [ sum(a.varm['scores']!=np.NINF) for a in annotated]
data['reads per cell'] = [ np.mean(np.sum(a.X,axis= 1)  ) for a in annotated]


table = pd.DataFrame(data)
table

In [ ]:
annotated[0].X[0].data

In [ ]:
if False:
    # another problem was that many cells could have too few reads 
    for name, ann in zip(names, annotated):
        print(name)
        plt.hist(np.sum(ann.X, axis = 1), bins = 75)
        plt.show()

In [ ]:
col = Zip(names, annot)
col = [c for c in col if c[1]>1500 ]
print ([c[0][:5] for c in col])
print(len(col))

# now make sure that 

In [ ]:
# so now we have some tissues with few representatives, lets clean this also


col = [c[0] for c in col] # list of names
a,b = np.unique([c[:5] for c in col ], return_counts=True)
itemcounts = Zip(a,b)
shitlist =  [item for item, count in itemcounts if count < 4]
flist = [c for c in col if c[:5] not in shitlist]


In [ ]:
flist

In [ ]:
import seaborn as sns 
import pandas as pd 

pdata = pd.DataFrame({'name':a,'count':b})
sns.barplot(data  = pdata,x='name',y='count')

In [ ]:

data['reads > 1400'] = [ sum(np.sum(a.X,axis= 1) > 1500 ) for a in annotated]


In [ ]:

import numpy as np

from scipy.sparse import csr_matrix as sparse

A = sparse([[1, 2, 0], [1, 0, 3], [4, 0, 5]])

In [ ]:
A.indptr

In [ ]:
acc = []
for ip in range(len(A.indptr)-1):
    start = A.indptr[ip]
    end = A.indptr[ip+1]
    #print(A.data[start:end])
    acc.append(A.indices[start:end])
    
np.array(acc)

In [ ]:


dist  = bla
a,b =dohugloop()
val = dists[a,b]

def repeated_linear_sum_assignment(distances, repeats):
        cc, rr  = [ do_a_round(dists) for range kneighbors  ].T
        return hstack cc , hstack rr
    
def doAround(distances):
        r, c = linear_sum_assignment(distances)
        distances [r,c] = inf
        return r,c 

In [ ]:
def findSliceNeighbors(data, distances, kNeighbors, indexStart):
        knnInd = np.empty((data.shape[0], 0), int)
        knnDists = np.empty((data.shape[0], 0), int)

        ### Gets 'k_neighbors' nearest neighbors from within the same timeslice
        i = 0
        while i < kNeighbors:
                rowInd, colInd = linear_sum_assignment(distances)
                knnDists = np.column_stack((knnDists, distances[rowInd, colInd]))
                distances, knnInd = calcNeighborIndices(distances, rowInd, colInd, knnInd, indexStart)

                i += 1   

        return knnInd, knnDists 